In [ ]:
# Standard Library
import argparse
import random
import pickle
from typing import Union

# Third-Party Libraries
import torch.optim as optim
from torch.utils.data import Dataset
import pandas as pd
import networkx as nx

# Custom Modules
from MatrixVectorizer import *
from dataloaders import NoisyDataset
from model import *
from preprocessing import *
from train import *
from evaluation import *

In [2]:
# Set a fixed random seed for reproducibility across multiple libraries
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

device = torch.device("cpu")

In [3]:
# load csvs as numpy
lr_data_path = './data/lr_train.csv'
hr_data_path = './data/hr_train.csv'

lr_train_data = pd.read_csv(lr_data_path, delimiter=',').to_numpy()
hr_train_data = pd.read_csv(hr_data_path, delimiter=',').to_numpy()
lr_train_data[lr_train_data < 0] = 0
np.nan_to_num(lr_train_data, copy=False)

hr_train_data[hr_train_data < 0] = 0
np.nan_to_num(hr_train_data, copy=False)

# map the anti-vectorize function to each row of the lr_train_data
lr_train_data_vectorized = torch.tensor([MatrixVectorizer.anti_vectorize(row, 160) for row in lr_train_data],
                                        dtype=torch.float32)
hr_train_data_vectorized = torch.tensor([MatrixVectorizer.anti_vectorize(row, 268) for row in hr_train_data],
                                        dtype=torch.float32)

splits, (lr_test_data,hr_test_data) = load_random_files(return_matrix=True)


/tmp/ipykernel_632366/3687639064.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  lr_train_data_vectorized = torch.tensor([MatrixVectorizer.anti_vectorize(row, 160) for row in lr_train_data],


In [4]:
train_data = NoisyDataset(lr_train_data_vectorized, hr_train_data_vectorized, noise_level=0.5)
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True)

In [8]:
# with comments showing the values for our grid search
num_splt = 3
epochs = 200
lr = 0.00005 # try [0.0001, 0.0005, 0.00001, 0.00005]
lmbda = 17 # should be around 15-20
lamdba_topo = 0.0005 # should be around 0.0001-0.001
lr_dim = 160
hr_dim = 320
hidden_dim = 320 # try smaller and larger - [160-512]
padding = 26
dropout = 0.2 # try [0., 0.1, 0.2, 0.3]


args = argparse.Namespace()
args.epochs = epochs
args.lr = lr
args.lmbda = lmbda
args.lamdba_topo = lamdba_topo
args.lr_dim = lr_dim
args.hr_dim = hr_dim
args.hidden_dim = hidden_dim
args.padding = padding
args.p = dropout


# Final Model & Kaggle Submission

In [9]:
#final train
ks = [0.9, 0.7, 0.6, 0.5]
criterion = nn.L1Loss()
scores = []

for lr_train_data, hr_train_data, lr_val_data, hr_val_data in splits:
    final_model = GSRNet(ks, args)
    final_model.to(device)
    optimizer = optim.Adam(final_model.parameters(), lr=args.lr)
    train_data = NoisyDataset(lr_train_data, hr_train_data, noise_level=0.5)
    train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True) 
    val_data = NoisyDataset(lr_val_data, hr_val_data, noise_level=0)
    train_val_loader = torch.utils.data.DataLoader(val_data, batch_size=1) 
    

    train(final_model, train_data_loader, optimizer, criterion, args)

    preds, loss = validate(final_model,train_val_loader, criterion, args)
    scores.append(evaluate_all(hr_val_data,preds))

print(scores)


Epoch:  1 Loss:  0.45927190227422887 Error:  0.20969157554432302 Topo:  36.410682860962645
Epoch:  2 Loss:  0.3102445595278711 Error:  0.18255760373469598 Topo:  20.425317592963488
Epoch:  3 Loss:  0.3022257846272634 Error:  0.17880466553622376 Topo:  19.412191602284324
Epoch:  4 Loss:  0.29694903450097865 Error:  0.17759281218408823 Topo:  19.119499400704207
Epoch:  5 Loss:  0.29097256385637615 Error:  0.17600396784122832 Topo:  18.684219988520272
Epoch:  6 Loss:  0.28645330560421517 Error:  0.17544859995742043 Topo:  18.958074090009678
Epoch:  7 Loss:  0.2823514826818854 Error:  0.17473038504580538 Topo:  18.991527511688048
Epoch:  8 Loss:  0.27717077509014904 Error:  0.17393922654097665 Topo:  18.707939056579225
Epoch:  9 Loss:  0.2734783270401869 Error:  0.17317594730568503 Topo:  18.943735282578153
Epoch:  10 Loss:  0.2698545825338649 Error:  0.17259178529242555 Topo:  18.894377245874463
Epoch:  11 Loss:  0.26590690150589286 Error:  0.17149784384730332 Topo:  18.638007775038304
Ep

In [ ]:
import pickle
filename = 'final-model.sav'
pickle.dump(final_model, open(filename, 'wb'))

In [ ]:
generate_submission_csv(final_model, args)